# About
Implement clustering algorithms
1. K-Means
1. Heirarchical clustering
1. DBSCAN

## The Team
| Name| Student ID|
|------------|---------------|
|Cynthia Cai | 5625483 |
|Pratyush Kumar | 5359252|


# Imports

// add the imports to the cell below

In [1]:
import numpy as np 
import pandas as pd
import scipy.spatial
from scipy.spatial import ConvexHull, distance_matrix
from sklearn.metrics.pairwise import euclidean_distances as eucDist
import glob
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

# Reading the dataset


From the readme for the xyz files, we know that:

Ground truth labels:
|File range|Label|
|--|--|
|    000 - 099: |building|
|    100 - 199: |car|
|    200 - 299: |fence|
|    300 - 399: |pole|
|    400 - 499: |tree|


workflow:

iterate through the files, and collect them in a dataframe

Use [this link](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat) for concatenating the dataframes

In [2]:
xyzPath = './scene_objects/data/*.xyz'

dataPathsList = glob.glob(xyzPath)

In [3]:
allPointsDF= pd.DataFrame(columns=['x','y','z', 'fileNo', 'groundLabel'])
# featureDF = pd.DataFrame(columns=['Label' , 'convHull', median] )

def df_maker(df1, df2):
    return pd.concat([df1, df2], sort=False, ignore_index=True)

labelToGive = None
for path in dataPathsList:
    indx = int(path.split('\\')[-1][0:3])
    # if else to determine label
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 

    # print(indx, labelToGive)        

    # using pandas to read dataset and make a dataFrame
    tempDF = pd.read_csv(path, delimiter=' ', header=None, dtype=np.float64, names=['x','y','z'])
    tempDF.loc[:,'fileNo'] = indx
    tempDF.loc[:,'groundLabel'] = labelToGive

    # merge with megaDFofPoints
    allPointsDF = df_maker(allPointsDF, tempDF)

# allPointsDF.head()

In [4]:
# save to pickle file
# allPointsDF.to_pickle('./scene_objects/compressedData.pkl')

## Making feature points
Identified feature points: `//add more`
* median height(z)
* convex hull

In [4]:
def label_determiner(indx):
    labelToGive=None
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 
    return labelToGive


featureDF = allPointsDF.groupby('fileNo').var()
featureDF.rename(columns={'x':'varX','y':'varY','z':'varZ'}, inplace=True)
featureDF.loc[:,'median_Z'] = allPointsDF.groupby('fileNo').z.median()
# featureDF.loc[:,'mean_Z'] = allPointsDF.groupby('fileNo').z.mean()

# range of x,y,z
featureDF.loc[:,'range_X'] = allPointsDF.groupby('fileNo').x.max() - allPointsDF.groupby('fileNo').x.min()
featureDF.loc[:,'range_Y'] = allPointsDF.groupby('fileNo').y.max() - allPointsDF.groupby('fileNo').y.min()
featureDF.loc[:,'range_Z'] = allPointsDF.groupby('fileNo').z.max() - allPointsDF.groupby('fileNo').z.min()

featureDF.loc[:,'Volume'] = allPointsDF.set_index('fileNo').loc[:,'x':'z'].groupby('fileNo').apply(ConvexHull).apply(lambda x: x.volume)

# points density
featureDF.loc[:,'footprintDensity'] =  allPointsDF.groupby('fileNo').count().x / (featureDF.range_X * featureDF.range_Y)
featureDF.loc[:,'volumeDensity'] =  allPointsDF.groupby('fileNo').count().x / featureDF.Volume

featureDF.loc[:,'label'] = featureDF.reset_index().fileNo.apply(label_determiner)

# standardize DF
standardFeatureDF = (featureDF.iloc[:,:-1] - featureDF.iloc[:,:-1].mean() ) / featureDF.iloc[:,:-1].std()

# join labels to the feature DF
standardFeatureDF = standardFeatureDF.join(other=featureDF.label ,on='fileNo')

featureDF.to_pickle('./scene_objects/featureData.pkl')
standardFeatureDF.to_pickle('./scene_objects/standardFeatureData.pkl')

featureDF

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
0,9.024868,1.760537,0.501052,17.92,10.480000,4.649994,5.02,104.189009,37.121449,17.362676,building
1,7.306054,2.628307,0.646622,7.53,10.540009,6.139984,3.60,137.366395,32.078876,15.112867,building
2,19.973520,18.707730,2.108935,13.35,17.039997,16.059998,7.49,1247.880682,32.711848,7.173763,building
3,27.224888,16.674539,2.437923,14.43,21.160004,16.750000,7.07,1326.712538,29.001490,7.747722,building
4,30.802399,22.456995,0.597981,7.80,23.579994,22.090012,5.21,1100.901866,23.277809,11.013697,building
...,...,...,...,...,...,...,...,...,...,...,...
495,2.468837,1.670906,2.625734,7.14,6.969986,5.820001,7.19,138.917876,67.520725,19.716685,tree
496,6.586047,4.938702,7.407274,9.84,10.640015,10.010002,12.44,645.231457,62.822416,10.369922,tree
497,1.379584,2.840469,2.370287,7.38,5.279999,6.920013,6.84,84.578072,31.994384,13.821549,tree


### Plotting to see resemblamces and clusters, if any
needed: seaborn

In [5]:
# load df's
featureDF = pd.read_pickle('./scene_objects/featureData.pkl')
standardFeatureDF = pd.read_pickle('./scene_objects/standardFeatureData.pkl')

In [ ]:
sns.pairplot(data=featureDF, hue="label")

normalize the feature df </br>
[from stackoverflow we see](https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame), that we can just use pandas for a standard scaling, or else, a [standard scaler from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) can also be applied </br>

from [answer here](https://stats.stackexchange.com/questions/417339/data-standardization-vs-normalization-for-clustering-analysis), we see that standard scaler is used for k means , so we are going with that

In [ ]:
sns.pairplot(data=standardFeatureDF, hue="label")

# Clustering Algorithms
note: already loaded the featureDF and standardised in the cell above

## K-Means clustering

In [7]:
def mean_vector(list):
    """
    Compute the mean value of each column of an array
    Input: an list of pts(array)
    Return: an array that represents a point
    """
    array = np.array(list) # an array(cluster) of array(pt)
    centroid = np.arange(len(array[0]),dtype=float) # an array(pt) of values (features)

    for i in range(0,len(array[0])):
        select_column = array[:,i] # returna an array
        # print("The selected column ",i,'is:',select_column)
        centroid[i] = np.mean(select_column)
    
    return centroid

In [ ]:

def kmeans(featureDF, k):
    """
    Use k-means algorithm to cluster feature points.

    Input:
        featureDF: dataframe that stores (500) feature points
        k: the number of clusters

    Output:
        C: a list of lists(cluster) of arrays(pt)
        C = [C1,C2,...,Ck]
    """
    #pre-step: setting parameters and format conversion
    MaxInteration, epsilon = 100, 3

    col_num = len(featureDF.columns.values.tolist()) # how many features
    pt_array = featureDF.to_numpy() # has 500 elements

    # Step1: initialize k centers
    centroid_index = np.random.randint(0,len(pt_array),k).tolist()
    #centroid_index = [50,150,250,350,450]
    centroid_array = featureDF.iloc[centroid_index].to_numpy() # has k elements

    # step2: for each feature point, find the nearest neighbour and assign cluster
    interation = 0
    while interation <= MaxInteration:
        interation += 1
        # initialize a bit
        old_centroid_array = centroid_array # store former centroids
        tree = scipy.spatial.cKDTree(centroid_array)
        cluster_list = [[]] * k # a list of lists(cluster) of arrays(pts)

        for pt in pt_array: # pt is a list
            dd, ii = tree.query(pt) # ii is the index of centroid
            if ii >= 0 and ii <= (k-1):
                cluster_list[ii].append(pt) # assign cluster(list)

        # step3: update centroids
        dist_list = [[]] * k
        for i in range(0,k):
            centroid_array[i] = mean_vector(cluster_list[i]) # the input is a list of arrays

            # step 4: compare old/new centroids and make interation judgement
            dist_list[i] = scipy.spatial.distance.euclidean(old_centroid_array[i],centroid_array[i])
        
        if max(dist_list) <= epsilon:
            break

    return cluster_list

kmeans_cluster_list = kmeans(featureDF,5)


## Heirarchical clustering

This [ref was nice](https://www.section.io/engineering-education/hierarchical-clustering-in-python/) for heirarchical clustering understanding
Some other sources:
* [Statquest](https://www.youtube.com/watch?v=7xHsRkOdVwo&ab_channel=StatQuestwithJoshStarmer)
* Penn state [pseudo code](https://online.stat.psu.edu/stat508/lesson/12/12.7)
* pseudo code from [researchgate](https://www.researchgate.net/figure/The-hierarchical-clustering-algorithm-in-pseudocode_fig1_202144697)
* towards data science article to do [step by step](https://towardsdatascience.com/breaking-down-the-agglomerative-clustering-process-1c367f74c7c2) {this is a good one to follow}
* another one [for theory](https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019)
* similar [theory as above](https://www.geeksforgeeks.org/ml-hierarchical-clustering-agglomerative-and-divisive-clustering/)
* real good [step by step explaination](https://medium.com/@darkprogrammerpb/agglomerative-hierarchial-clustering-from-scratch-ec50e14c3826), also the [github code](https://github.com/Darkprogrammerpb/DeepLearningProjects/blob/master/Project40/agglomerative_hierarchial_clustering/Hierarchial%20Agglomerative%20clustering.ipynb)

### To Think in heirarchical clustering:
* Which type of heirarchical clustering are we doing: lets begin with agglomerative clustering
* Within the selected type what distance metrics are we using


In [ ]:

tempDF = standardFeatureDF.iloc[:,:-1].copy()

def heirarch_clust(dataDF):
    distances = eucDist(standardFeatureDF.drop('label', axis=1))
    
    pass


# calculate distances
# maybe change the distance computation
distMatDF = pd.DataFrame( distance_matrix(tempDF.values, tempDF.values), index = tempDF.index, columns = tempDF.index)
# distMatDF = pd.DataFrame( np.tril(distMatDF),  index = tempDF.index, columns = tempDF.index)
distMatDF = distMatDF.where(distMatDF!=0, np.nan)
distMatDF


devise new distance matrix and then repeat the sequence:
### TODO: 
* linkage between the clusters
* updation of the distance matrix

clusters to be made:
`vals.idxmin()` and `idVals.iloc[vals.idxmin()]`

In [4]:
tempDF = standardFeatureDF.iloc[:,:-1].copy()

distMatDF = pd.DataFrame( distance_matrix(tempDF.values, tempDF.values), index = tempDF.index, columns = tempDF.index)
# distMatDF = pd.DataFrame( np.tril(distMatDF),  index = tempDF.index, columns = tempDF.index)
# replace 0 distances with np.nan
distMatDF = distMatDF.where(distMatDF!=0, np.nan)
    
clusterKeeper = {}
clustDict={}
clusterKeeperList = []
clustCheck = {}
# clustCHECK WILL have two nodes each
iterationCounter=0
play=[]
m=len(distMatDF)
progression = [ [i] for i in range(m) ] 

while m>1: 

    # cluster size
    # print(f"Total sample = {m}")
    # compute distances

    # get indices with min dist
    vals = distMatDF.min(skipna=True)
    idVals = distMatDF.idxmin(skipna=True)

    # print(vals.min(), vals.idxmin()) # GIVES US THE MINIMUM VALUE and the index at which this was found in the vals series
    # print(idVals.iloc[vals.idxmin()])
    
    ind_to_pop = [idVals.loc[vals.idxmin()] , vals.idxmin()]
    # print(f"index {ind_to_pop}")
    play.append(ind_to_pop)
    # update distmatrix at some point
    # add updated new row, col to dist mat  
    # this updated row is basically the minimum of the two eliminated rows
    singleLink_minRow = distMatDF.loc[ind_to_pop].drop(ind_to_pop, axis=1).max()
    singleLink_minRow.rename(f"cluster {iterationCounter}", inplace=True)

    # pop row and col from dist mat
    distMatDF = distMatDF.drop(ind_to_pop, axis=0).drop(ind_to_pop, axis=1)
    # print("row,col ",len(distMatDF),len(distMatDF.columns))

    # min distance from other points

    distMatDF = distMatDF.append(singleLink_minRow)
    distMatDF.loc[:,singleLink_minRow.name] = singleLink_minRow
    # update value of m
    m = len(distMatDF)
    # m-=1
    clusterKeeper[f"iteration {iterationCounter}"] = {'indices_popped':ind_to_pop , "df":distMatDF.copy()}
    clusterKeeperList.append( (iterationCounter, ind_to_pop) )
    clustDict[f"cluster {iterationCounter}"] = ind_to_pop
    
    indPop1, indPop2 = ind_to_pop

    clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":indPop2, 'fullnodes':ind_to_pop}
    print("before" , clustCheck[f'cluster {iterationCounter}'])
    
    # Case: if first index is a cluster
    if (indPop1 in clustCheck.keys()) and (indPop2 in clustCheck.keys()): #both are clusters
        clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":clustCheck[indPop2]['fullnodes'].copy() }
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        # try:
        tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy()) #if it is a list
        # except:
        #     tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"]) # if it isnt a list and thus can't be copied
        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull  


    # Case: if first index is a cluster
    elif indPop1 in clustCheck.keys(): #means first position is cluster
        clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":indPop2 }
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        try:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy()) #if it is a list
        except:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"]) # if it isnt a list and thus can't be copied
        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull

    # Case: if second index is a cluster
    elif indPop2 in clustCheck.keys(): #means first position is cluster
        clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":clustCheck[indPop2]['fullnodes'].copy()}
        tempFull = clustCheck[f"cluster {iterationCounter}"]["node1"].copy()
        try:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"].copy())
        except:
            tempFull.append(clustCheck[f"cluster {iterationCounter}"]["node2"])

        clustCheck[f"cluster {iterationCounter}"]["fullnodes"] =  tempFull

    print("after" , clustCheck[f'cluster {iterationCounter}'])

    iterationCounter+=1
distMatDF

before {'node1': 151, 'node2': 142, 'fullnodes': [151, 142]}
after {'node1': 151, 'node2': 142, 'fullnodes': [151, 142]}
before {'node1': 198, 'node2': 123, 'fullnodes': [198, 123]}
after {'node1': 198, 'node2': 123, 'fullnodes': [198, 123]}
before {'node1': 160, 'node2': 136, 'fullnodes': [160, 136]}
after {'node1': 160, 'node2': 136, 'fullnodes': [160, 136]}
before {'node1': 389, 'node2': 370, 'fullnodes': [389, 370]}
after {'node1': 389, 'node2': 370, 'fullnodes': [389, 370]}
before {'node1': 148, 'node2': 135, 'fullnodes': [148, 135]}
after {'node1': 148, 'node2': 135, 'fullnodes': [148, 135]}
before {'node1': 175, 'node2': 167, 'fullnodes': [175, 167]}
after {'node1': 175, 'node2': 167, 'fullnodes': [175, 167]}
before {'node1': 96, 'node2': 83, 'fullnodes': [96, 83]}
after {'node1': 96, 'node2': 83, 'fullnodes': [96, 83]}
before {'node1': 194, 'node2': 103, 'fullnodes': [194, 103]}
after {'node1': 194, 'node2': 103, 'fullnodes': [194, 103]}
before {'node1': 185, 'node2': 104, 'ful

fileNo,cluster 498
fileNo,
cluster 498,NaN


## DBSCAN

In [21]:
import random

# pre-step: data structure and kdtree
radius, MinPts = 0.8, 4

column_name = standardFeatureDF.columns.values.tolist()
pts_label_list = standardFeatureDF.to_numpy().tolist()
print('column',column_name)

pts_DF = standardFeatureDF.loc[:,'varX':'volumeDensity']
pts_array = pts_DF.to_numpy() # an array of arrays
tree = scipy.spatial.cKDTree(pts_array)

# step1: generate list of core points (with MinPts)
wait_idx = [*range(0,len(pts_DF))]
core_idx = [] # a list of lists (core pt) !!!可以存的是wait_list的index
for i in wait_idx:
    pt = pts_array[i]
    neighbor = tree.query_ball_point(pt, radius) # a list of indexs of neighbors
    # print('neighbor',neighbor,type(neighbor))
    if len(neighbor) >= MinPts:
        core_idx.append(i) # What about index? We need it for label.

print("core point num:",len(core_idx))
print("whole point num:",len(wait_idx))

column ['varX', 'varY', 'varZ', 'median_Z', 'range_X', 'range_Y', 'range_Z', 'Volume', 'footprintDensity', 'volumeDensity', 'label']
core point num: 345
whole point num: 500


In [22]:
# step2:
# initialize cluster number and waiting list
k = 0
cluster = []

print("core index:",core_idx)

while len(core_idx) > 0:
    clusterDF = pd.DataFrame(columns=column_name)
    
    # pick a random core point and generate the loop
    start_idx = random.sample(core_idx,1)[0]
    queue = [start_idx]
    core_idx.remove(start_idx)
    wait_idx.remove(start_idx)
    clusterDF.append(pts_label_list[start_idx])
    #print("start index:",start_idx)

    while len(queue) > 0:
        # process each pt in queue
        # first of all: core or non-core?
        # for a non-core point, assign it to a cluster (done)
        # for a core neighbor point, assign it to a cluster (done) and add its neighbors to queue
        pt = pts_array[queue.pop(0)] 
        neighbor = tree.query_ball_point(pt, radius) # a list of neighbor indexes        
        
        if len(neighbor) >= MinPts:
            for index in neighbor:
                if index in wait_idx:
                    queue.append(index) # 将 在邻域中 & 未处理 的点 加入 队列
                    wait_idx.remove(index) # 将 在邻域中 & 未处理 的点 移除出 等待集
                    clusterDF.loc[len(clusterDF.index)] = pts_label_list[index] # 将 在邻域中 & 未处理 的点 加入聚类簇
                    if index in core_idx:
                        core_idx.remove(index) # 将 在邻域中 & 未处理 的点 移除出 核心集
    
    print("The {} cluster: {}\n".format(k,clusterDF)) 

    # store the new cluster
    k = k + 1
    cluster.append(clusterDF)

    if k > 10:
        break


core index: [1, 2, 3, 5, 6, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 24, 26, 27, 28, 30, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 50, 51, 53, 56, 57, 58, 59, 61, 62, 63, 65, 68, 70, 71, 75, 78, 80, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 95, 96, 99, 100, 101, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 205, 206, 207, 209, 211, 214, 218, 219, 221, 222, 223, 225, 228, 229, 230, 231, 232, 233, 235, 236, 237, 238, 239, 241, 242, 243, 244, 249, 250, 253, 254, 255, 259, 260, 262, 263, 264, 267, 268, 269, 270, 271, 272, 273, 275, 276, 277, 279, 280, 2

In [42]:
# step2:
# initialize cluster number and waiting list
k = 0
cluster = []

while len(core_pts) > 0:
    cluster_pts = [] # a list of lists (pts)
    # initilize the queue with a random core point
    # reference: https://www.geeksforgeeks.org/python-random-sample-function/
    #random_pt = core_list[random.randrange(len(core_list))] # or: random.sample(list(data), k) ; an array
    start_pt = random.sample(core_pts,1)[0]

    #random_index = random.randrange(len(core_list))
    #random_pt = core_list[random_index]

    queue = [start_pt] # 或许用Index来Queue! 
    core_pts.remove(start_pt)
    wait_pts.remove(start_pt)
    cluster_pts.append(start_pt)

    while len(queue) > 0:
        pt = queue.pop(0) # reference: https://www.programiz.com/python-programming/methods/list/pop
        neighbor = tree.query_ball_point(pt, radius) # array of lists (pt) # 这里是pt的形式
        print("The neighbor points: ",neighbor)

        # 将 在邻域中 & 未处理 的点 加入 队列
        # 将 在邻域中 & 未处理 的点 移除出 等待集 
        # Non-core points can be assigned to a cluster, but cann not expand border.
        if len(neighbor) >= MinPts:
            for value in neighbor:
                if value in wait_pts:
                    queue.append(value)
                    wait_pts.remove(value)
                    cluster_pts.append(value)
                    if value in core_pts:
                        core_pts.remove(value)
        
    # generate a new cluster
    k = k + 1
    cluster.append(cluster_pts)


# Validation

In [ ]:
def validateModels(classifiedData, originalData):
    pass